In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup


# 웹드라이버 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 디버거 모드로 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=webdriver.ChromeOptions().add_experimental_option("debuggerAddress","localhost:9222"))

site_url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%9E%90%EB%8F%99%EC%B0%A8&oquery=%EC%9E%90%EB%8F%99%EC%B0%A8&tqi=i6QNUwprvmZssfTsc00ssssssOK-244827'
driver.get(site_url)

hrefs = set()

while True:
    # 현재 페이지의 HTML을 가져옴
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 'div.info_box' 내의 'a' 태그를 찾아서 href 속성 저장
    a_tags = soup.select('div.info_box a')
    urls = [a['href'] for a in a_tags if 'href' in a.attrs]
    new_hrefs = set(urls) - hrefs  # 이전에 가져온 href 정보와 중복된 것을 제외함
    hrefs.update(new_hrefs)  # 새로운 href 정보를 추가

    # 다음 페이지로 넘어감
    try:
        next_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//a[@class="pg_next on"]')))
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(0.1)
    except:
        break

In [ ]:
base_url = 'https://search.naver.com/search.naver?'

def parse_car_name_kind(html):
    # 이름, 사진 갖고오기
    # soup = BeautifulSoup(html, 'html.parser')
    soup = BeautifulSoup(html, 'lxml')
    car_name = soup.select('span.area_text_title ._text')[0].text.strip()
    car_img = soup.find("div", class_="img_area").find("img").get("src")
    return  car_name, car_img


In [ ]:
import csv
from tqdm import tqdm

with open("./car_img.csv", "w", newline="", encoding="utf-8-sig") as fw:
    writer = csv.writer(fw)
    headers = ["name", "img"]
    writer.writerow(headers)

    for href in tqdm(hrefs):
        full_url = base_url + href
        driver.get(full_url)
        car_name, car_img = parse_car_name_kind(driver.page_source)
        car_info = [car_name, car_img]
        writer.writerow(car_info)